# 基于 First Order Model 的 damedane/unravel 教程

使用方法：点击上方菜单，选择 Run（运行） - Run All Cells（运行所有）

如果需要修改输入视频和音频的名称，请访问 [7] 代码块，修改其中的 `source_image_name` 和 `source_video_name` 变量，并再运行 [7] 以及之后的所有代码块

教程文章说明请查看 [openbayes.blog/259](https://openbayes.blog/259)

In [1]:
!python --version
!pip --version

Python 3.6.12 :: Anaconda, Inc.
pip 20.3.3 from /usr/local/lib/python3.6/site-packages/pip (python 3.6)


In [2]:
cd '/openbayes/home/first-order-model/'

/output/first-order-model


In [3]:
!git pull

Already up to date.


In [4]:
cd /openbayes/home

/output


In [5]:
!ls -lah

total 24M
drwxr-xr-x  5 root root  4.0K Mar  5 08:09 .
drwxr-xr-x  1 root root  4.0K Mar  5 08:04 ..
-rw-r--r--  1  501 staff  461 Dec 19  2019 ._openbayes-intro.ipynb
drwxr-xr-x  2 root root  4.0K Sep  1  2020 .ipynb_checkpoints
drwxr-xr-x  2 root root  4.0K Mar  5 07:40 .pylibs
-rw-r--r--  1 root root  555K Mar  5 08:09 damedane.ipynb
drwxr-xr-x 10 root root  4.0K Aug 29  2020 first-order-model
-rw-r--r--  1 root root  1.1M Aug 29  2020 import.mp4
-rw-r--r--  1 root root   82K Aug 29  2020 import.png
-rw-r--r--  1  501 staff 485K Dec 19  2019 openbayes-intro.ipynb
-rw-r--r--  1 root root   19M Sep  4 16:09 unravel.mp4
-rw-r--r--  1 root root  2.8M Mar  5 07:42 蚂蚁呀嘿.mp4


In [6]:
pip install imageio-ffmpeg ffmpeg-python

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [7]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

# 下面修改输入图片的文件名
source_image_name = 'import.png'

# 下面修改输入视频的文件名
source_video_name = '蚂蚁呀嘿.mp4'

source_image = imageio.imread(source_image_name)
driving_video = imageio.get_reader(source_video_name)


# 调整图片和视频分辨率为 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

HTML(display(source_image, driving_video).to_html5_video())

In [8]:
import sys
sys.path.append('/openbayes/home/first-order-model')

In [9]:
cd /openbayes/home/

/output


In [10]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='first-order-model/config/vox-256.yaml', 
                            checkpoint_path='/openbayes/input/input0/vox-cpk.pth.tar')
                            # 加载配置文件并加载数据集

In [11]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

100%|██████████| 191/191 [00:04<00:00, 38.88it/s]


In [12]:
import cv2
current_fps = cv2.VideoCapture(source_video_name).get(cv2.CAP_PROP_FPS)

# 保存结果
imageio.mimsave('export-no-audio.mp4', [img_as_ubyte(frame) for frame in predictions], fps=current_fps)

# 生成预览
HTML(display(source_image, driving_video, predictions).to_html5_video())

In [13]:
import ffmpeg

# 合并音频 + 视频
video = ffmpeg.input('export-no-audio.mp4')
audio = ffmpeg.input(source_video_name)

ffmpeg.concat(video, audio, v=1, a=1).output('export.mp4').overwrite_output().run()
print("输出完成，右键点击左侧 export.mp4 文件选择 Download 进行下载\n执行完后记得随手关闭容器，否则算力资源会继续计费！")

输出完成，右键点击左侧 export.mp4 文件选择 Download 进行下载
执行完后记得随手关闭容器，否则算力资源会继续计费！
